In [8]:
from nltk.corpus import gutenberg
from nltk.model import *
from nltk.probability import LidstoneProbDist
import pprint
import sys

class NullDevice():
    def write(self, s):
        pass

use_chars = True

#sys.stderr = NullDevice()
estimator = lambda fdist, bins: LidstoneProbDist(fdist, 0.02)

t = None
if use_chars:
    t = [ [ c for c in ' '.join(sent) ] for sent in gutenberg.sents('carroll-alice.txt') ]
else:
    t = [ x for x in gutenberg.sents('carroll-alice.txt') ]

def ngram_gen(n, t, len=100):
    print "\n%d-gram model\n---------------\n" % (n)
    m = NgramModel(n, t, True, True, estimator)
    #print m.entropy("if you cannot mean what you say then at least say what you mean".split())
    sep = '' if use_chars else ' '
    if n == 1:
        # there is a bug in nltk NgramModel when n=1. it just produces a sequence of . characters
        genstring = sep.join([ m[()].generate() for i in range(len) ])
    else:
        genstring = sep.join(m.generate(len))
    print genstring
    return genstring

genstring_list = map(lambda n: ngram_gen(n, t), range(1,10))



1-gram model
---------------

 apoA itb  ai g WcHTdi  cnngmoduwFt ,ieiwsei !epe i eyhn, l  ps''tAqif wearw i e ot   n Yl tda'e eri

2-gram model
---------------

'' we ' thtthish sherlozeve ars Fope Allid heatowoutorye me aleathe lad , and Mofraly , OCrg ll t f

3-gram model
---------------

Shesten .'' shent to hat was nockind wrep as eyethe ple MING Alithead upok hat whe Haremble .'

4-gram model
---------------

She Duch a VERY unease said it deciden a bird began older , wentily such they ' You were , poing to 

5-gram model
---------------

' crumbling it would , and as in a thing , ' Pray in he repeaten on sea !' exclamourn ; so make 

6-gram model
---------------

' Oh , YOU like the fifteenth , as there , you been .''These would this time whom she

7-gram model
---------------

And yet you if you ' re looked up eager with one flapper across the first witness , she heads off .

8-gram model
---------------

As for pulling me out among the people about her hand , in the middle , 

In [12]:
# from Roger Levy
# http://idiom.ucsd.edu/~rlevy/teaching/2015winter/lign165/code/NgramModel.py
# http://idiom.ucsd.edu/~rlevy/teaching/2015winter/lign165/lectures/lecture11/lecture11_ngrams_in_Python.pdf

import nltk, random, collections, math
from math import log
from collections import Counter
from nltk.corpus import gutenberg

use_chars = False
# we can get replicable behavior by initializing the random number generator with random.seed()
random.seed(1)

def flatten_once(listOfListsOfLists):
    result = []
    for x in listOfListsOfLists:
        result = result + x
    return(result)
    
#documents = [list(movie_reviews.sents(fileid))
#             for fileid in movie_reviews.fileids()]
#random.shuffle(documents)
#train,dev,test = flatten_once(documents[0:1800]),flatten_once(documents[1800:1900]),flatten_once(documents[1900:2000])

train = None
if use_chars:
    train = [ [ c for c in ' '.join(sent) ] for sent in gutenberg.sents('carroll-alice.txt') ]
else:
    train = [ x for x in gutenberg.sents('carroll-alice.txt') ]


boundaryToken = "</s>"
def ngrams(n,sentences,boundaryToken=boundaryToken,verbose=False):
    c = {}
    q = []
    for i in range(n-1):
        q.append(boundaryToken)
    for sentence in sentences:
        for w in sentence + [boundaryToken]:
            context_gram = stringify_context(q)
            if verbose:
                print(q)
                print(context_gram)
                print(w)
            if not context_gram in c:
                c[context_gram] = Counter()
            c[context_gram][w] += 1
            q.pop(0)
            q.append(w)
    return(c)


def stringify_context(context):
    return(" ".join(context))
    
class BigramModel:
    def __init__(self, training_sentences, smoothing='none'):
        train = ngrams(2,training_sentences)
        self.probs = {}
        if smoothing=='none':
            for context_gram in train.keys():
                N = sum(train[context_gram].values())
                self.probs[context_gram] = Counter({k:v/N for k,v in train[context_gram].items()})

    def prob(self,word,context):
        """takes a word string and a context which is a list of word strings, and returns the probability of the word"""
        c = stringify_context(context)
        return(self.probs[c][word])

    def scoreSentence(self,sentence,verbose=False):
        context = [boundaryToken]
        result = 0
        for w in sentence + [boundaryToken]:
            lp = log(self.prob(w,context))
            result = result + lp
            if verbose:
                print(context,w,lp)
            context = [w]
        return result

    def generateSentence(self,verbose=False,goryDetails=False):
        context = [boundaryToken]
        result = []
        w = None
        while not w == boundaryToken:
            r = random.random() # returns a random float between 0 and 1
            x = 0
            c = self.probs[stringify_context(context)] # this will be a Counter
            for k,v in c.items():
                x = x + v
                if goryDetails:
                    print(r,context,x,k,v)
                if x > r: # choose this word
                    w = k
                    result.append(w)
                    context = [w]
                    break
            if verbose:
                print(w)
        result.pop() # drop the boundary token
        return result

for i in train[3:6]:
    print " ".join(i)
ngrams(2,train,boundaryToken="</s>",verbose=True)
#m = BigramModel(train)
#test_sentence = ['the','white','rabbit']
#m.scoreSentence(test_sentence,verbose=True)
#m.generateSentence(verbose=True)

Alice was beginning to get very tired of sitting by her sister on the bank , and of having nothing to do : once or twice she had peeped into the book her sister was reading , but it had no pictures or conversations in it , ' and what is the use of a book ,' thought Alice ' without pictures or conversation ?'
So she was considering in her own mind ( as well as she could , for the hot day made her feel very sleepy and stupid ), whether the pleasure of making a daisy - chain would be worth the trouble of getting up and picking the daisies , when suddenly a White Rabbit with pink eyes ran close by her .
There was nothing so VERY remarkable in that ; nor did Alice think it so VERY much out of the way to hear the Rabbit say to itself , ' Oh dear !
['</s>']
</s>
[
[u'[']
[
Alice
[u'Alice']
Alice
'
[u"'"]
'
s
[u's']
s
Adventures
[u'Adventures']
Adventures
in
[u'in']
in
Wonderland
[u'Wonderland']
Wonderland
by
[u'by']
by
Lewis
[u'Lewis']
Lewis
Carroll
[u'Carroll']
Carroll
1865
[u'1865']
1865
]


{u'secondly': Counter({u',': 2}),
 u'pardon': Counter({u"!'": 4, u',': 1, u",'": 1}),
 u'Tut': Counter({u',': 1}),
 u'saves': Counter({u'a': 1}),
 u'knelt': Counter({u'down': 1}),
 u'four': Counter({u'feet': 1,
          u'hours': 2,
          u'inches': 1,
          u'thousand': 1,
          u'times': 3}),
 u'Does': Counter({u'YOUR': 1, u'the': 1}),
 u'sleep': Counter({u'"': 1,
          u"'": 1,
          u',': 1,
          u'these': 1,
          u'when': 1,
          u'you': 1}),
 u'hanging': Counter({u'down': 1, u'from': 1, u'out': 1}),
 u'ringlets': Counter({u',': 1, u'at': 1}),
 u'oldest': Counter({u'rule': 1}),
 u'hate': Counter({u'--': 1, u'cats': 1}),
 u'assembled': Counter({u'about': 1, u'on': 1}),
 u'consider': Counter({u'their': 1}),
 u'whose': Counter({u'cause': 1, u'thoughts': 1}),
 u'swam': Counter({u'about': 1,
          u'lazily': 1,
          u'nearer': 1,
          u'slowly': 1,
          u'to': 1}),
 u'YOURS': Counter({u':': 1}),
 u'presents': Counter({u'like': 1, u

In [ ]:
from IPython.core.display import HTML


def css_styling():
    styles = open("../css/notebook.css", "r").read()
    return HTML(styles)
css_styling()